In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torchvision.models import resnet18
import torchvision
import torchvision.transforms as transforms
import time
import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning)

In [6]:
device = "cuda" if torch.cuda.is_available() else "cpu"

transform = transforms.Compose([
    transforms.RandomCrop(32, padding=4),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    transforms.Normalize((0.5,), (0.5,))
])

trainset = torchvision.datasets.CIFAR10(root='./data', train=True, download=True, transform=transform)
trainloader = torch.utils.data.DataLoader(trainset, batch_size=128, shuffle=True, num_workers=2)

testset = torchvision.datasets.CIFAR10(root='./data', train=False, download=True, transform=transform)
testloader = torch.utils.data.DataLoader(testset, batch_size=128, shuffle=False, num_workers=2)

In [7]:
class Bottleneck(nn.Module):
    def __init__(self, in_ch, out_ch=256):
        super().__init__()
        self.conv = nn.Conv2d(in_ch, out_ch, 1)
        self.bn = nn.BatchNorm2d(out_ch)

    def forward(self, x):
        return self.bn(self.conv(x))

class SelfDistillResNet18(nn.Module):
    def __init__(self, num_classes=10):
        super().__init__()
        base = resnet18(pretrained=False)

        self.stem = nn.Sequential(
            nn.Conv2d(3, 64, 3, padding=1, bias=False),
            nn.BatchNorm2d(64),
            nn.ReLU(inplace=True)
        )

        self.layer1 = base.layer1 # 64
        self.layer2 = base.layer2 # 128
        self.layer3 = base.layer3 # 256
        self.layer4 = base.layer4 # 512

        # Bottlenecks
        self.b1 = Bottleneck(64, 256)
        self.b2 = Bottleneck(128, 256)
        self.b3 = Bottleneck(256, 256)
        self.b4 = Bottleneck(512, 256)

        # Classifiers
        self.fc1 = nn.Linear(256, num_classes)
        self.fc2 = nn.Linear(256, num_classes)
        self.fc3 = nn.Linear(256, num_classes)
        self.fc4 = nn.Linear(256, num_classes)

    def forward(self, x):
        x = self.stem(x)

        f1 = self.layer1(x)
        h1 = self.b1(f1)
        p1 = self.fc1(F.adaptive_avg_pool2d(h1,1).flatten(1))

        f2 = self.layer2(f1)
        h2 = self.b2(f2)
        p2 = self.fc2(F.adaptive_avg_pool2d(h2,1).flatten(1))

        f3 = self.layer3(f2)
        h3 = self.b3(f3)
        p3 = self.fc3(F.adaptive_avg_pool2d(h3,1).flatten(1))

        f4 = self.layer4(f3)
        h4 = self.b4(f4)
        p4 = self.fc4(F.adaptive_avg_pool2d(h4,1).flatten(1))

        return [p1, p2, p3, p4], [h1, h2, h3, h4]

In [8]:
CE = nn.CrossEntropyLoss()
KL = nn.KLDivLoss(reduction="batchmean")
MSE = nn.MSELoss()

T = 4.0
alpha = 0.7
beta = 0.3

def train_step(model, images, labels, optimizer):
    logits, feats = model(images)

    teacher_logits = logits[-1]
    teacher_feat = feats[-1].detach()

    total_loss = 0

    for i in range(4):
        student_logits = logits[i]
        student_feat = feats[i]

        loss_ce = CE(student_logits, labels)

        if i == 3:
            total_loss += loss_ce
            continue

        log_p = F.log_softmax(student_logits / T, dim=1)
        q = F.softmax(teacher_logits / T, dim=1)
        loss_kl = KL(log_p, q) * (T*T)

        teacher_resized = F.interpolate(teacher_feat, size=student_feat.shape[2:], mode="bilinear")
        loss_l2 = MSE(student_feat, teacher_resized)

        total_loss += loss_ce + alpha * loss_kl + beta * loss_l2

    optimizer.zero_grad()
    total_loss.backward()
    optimizer.step()

    return total_loss.item()

In [11]:
lr = 0.05
epochs = 50
model = SelfDistillResNet18().to(device)

optimizer = torch.optim.SGD(model.parameters(), lr=lr, momentum=0.9, weight_decay=5e-4)
scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=epochs)

start_time = time.time()

for epoch in range(epochs):

    model.train()
    total_epoch_loss = 0
    for images, labels in trainloader:
        images, labels = images.to(device), labels.to(device)
        loss = train_step(model, images, labels, optimizer)
        total_epoch_loss += loss

    avg_loss = total_epoch_loss / len(trainloader)

    scheduler.step()
    model.eval()
    correct = 0
    total = 0
    with torch.no_grad():
        for images, labels in testloader:
            images, labels = images.to(device), labels.to(device)
            logits, _ = model(images)
            final_outputs = logits[-1]
            _, predicted = torch.max(final_outputs, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
    acc = 100 * correct / total

    current_lr = optimizer.param_groups[0]['lr']
    print(f"Epoch {epoch+1} | Current lr: {current_lr:.4f} | Loss: {avg_loss:.4f} | Accuracy: {acc:.2f}%")

total_time = time.time() - start_time
print(f"\nTotal training time: {total_time:.2f} seconds")

torch.save(model.state_dict(), "saved_models/cifar_10.pt")
print("Model saved as saved_models/cifar_10.pt")


Epoch 1 | Current lr: 0.0500 | Loss: 6.8106 | Accuracy: 52.40%
Epoch 2 | Current lr: 0.0498 | Loss: 4.7885 | Accuracy: 62.03%
Epoch 3 | Current lr: 0.0496 | Loss: 4.0486 | Accuracy: 68.93%
Epoch 4 | Current lr: 0.0492 | Loss: 3.5853 | Accuracy: 63.47%
Epoch 5 | Current lr: 0.0488 | Loss: 3.2461 | Accuracy: 65.46%
Epoch 6 | Current lr: 0.0482 | Loss: 3.0227 | Accuracy: 75.48%
Epoch 7 | Current lr: 0.0476 | Loss: 2.8167 | Accuracy: 76.88%
Epoch 8 | Current lr: 0.0469 | Loss: 2.6839 | Accuracy: 79.67%
Epoch 9 | Current lr: 0.0461 | Loss: 2.5615 | Accuracy: 68.75%
Epoch 10 | Current lr: 0.0452 | Loss: 2.4708 | Accuracy: 80.16%
Epoch 11 | Current lr: 0.0443 | Loss: 2.3898 | Accuracy: 80.42%
Epoch 12 | Current lr: 0.0432 | Loss: 2.3036 | Accuracy: 81.29%
Epoch 13 | Current lr: 0.0421 | Loss: 2.2411 | Accuracy: 76.83%
Epoch 14 | Current lr: 0.0409 | Loss: 2.1804 | Accuracy: 81.09%
Epoch 15 | Current lr: 0.0397 | Loss: 2.1125 | Accuracy: 80.46%
Epoch 16 | Current lr: 0.0384 | Loss: 2.0609 | Ac